# Team F1 Data Wrangling Master 

-Goals and Objectives:
1. Mold the Results csv into basic shape
2. Add in all required columns
3. address all N/A's
4. Save as a separate CSV
5. Enact test/train split

In [3]:
import numpy as np
import pandas as pd
from collections import Counter

## Import statements for our CSV's

In [4]:
#results
results_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/results.csv'
results_df = pd.read_csv(results_url, sep = ',', engine = 'python')
results_df.head()

#status
status_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/status.csv'
status_df = pd.read_csv(status_url, sep = ',', engine = 'python')

#circuits
circuits_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/circuits.csv'
circuits_df = pd.read_csv(circuits_url, sep = ',', encoding = 'latin-1')

#races
races_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/races.csv'
races_df = pd.read_csv(races_url, sep = ',', engine = 'c')

#Drivers
drivers_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/drivers.csv'
drivers_df = pd.read_csv(drivers_url, sep = ',', encoding = 'latin-1')

#lap Times
lap_times_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/lap_times.csv'
lap_times_df = pd.read_csv(lap_times_url, sep = ',', engine = 'python')

#MasterData1
MasterData1_url = 'https://raw.githubusercontent.com/georgetown-analytics/Formula1/main/data/MasterData1.csv'
MasterData1_df = pd.read_csv(MasterData1_url, sep = ',', engine = 'python')

In [6]:
races_df.describe()

,raceId,year,round,circuitId
count,1058.000000,1058.000000,1058.000000,1058.000000
mean,531.231569,1990.780718,8.382798,22.191871
std,308.165709,19.730088,5.000281,17.294490
min,1.000000,1950.000000,1.000000,1.000000
25%,265.250000,1976.000000,4.000000,9.000000
50%,529.500000,1992.000000,8.000000,18.000000
75%,793.750000,2008.000000,12.000000,32.000000
max,1073.000000,2021.000000,23.000000,77.000000


## Molding Results.csv into basic shape

### Tasks
1. drop:
    1. resultID
    2. number
    3. points
    4. time
    5. fastestLapTime

In [26]:
results_df = results_df.drop(['resultId','number','points','time','fastestLapTime'], axis=1, inplace=False)
results_df.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,milliseconds,fastestLap,rank,fastestLapSpeed,statusId
0,18,1,1,1,1,1,1,58,5690616,39,2,218.300,1
1,18,2,2,5,2,2,2,58,5696094,41,3,217.586,1
2,18,3,3,7,3,3,3,58,5698779,41,5,216.719,1
3,18,4,4,11,4,4,4,58,5707797,58,7,215.464,1
4,18,5,1,3,5,5,5,58,5708630,43,1,218.385,1


## Adding Required Columns

### Tasks:
1. Add Race Outcome from status.csv
2. Add FamilyID from status.csv
3. Add circuitType from circuits.csv
4. Add isHistoric from circuits.csv
5. Add Driver Place of Origin

#### #1 and #2
Adding race outcome and familyID from status.csv

In [27]:
merged2_df = pd.merge(results_df,status_df,on='statusId')
#note - I did this with just an inner join because we know for sure that there are no nulls in statusID

In [89]:
merged2_df.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,milliseconds,fastestLap,rank,fastestLapSpeed,statusId,status,familyStatus,Completion Status
0,18,1,1,1,1,1,1,58,5690616,39,2,218.300,1,Finished,4,1
1,18,2,2,5,2,2,2,58,5696094,41,3,217.586,1,Finished,4,1
2,18,3,3,7,3,3,3,58,5698779,41,5,216.719,1,Finished,4,1
3,18,4,4,11,4,4,4,58,5707797,58,7,215.464,1,Finished,4,1
4,18,5,1,3,5,5,5,58,5708630,43,1,218.385,1,Finished,4,1


In [28]:
#get rid of the leftovers on merged2
merged2_df = merged2_df.drop(columns=['status','statusId'],axis=1,inplace=False)

In [91]:
merged2_df

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,milliseconds,fastestLap,rank,fastestLapSpeed,familyStatus,Completion Status
0,18,1,1,1,1,1,1,58,5690616,39,2,218.300,4,1
1,18,2,2,5,2,2,2,58,5696094,41,3,217.586,4,1
2,18,3,3,7,3,3,3,58,5698779,41,5,216.719,4,1
3,18,4,4,11,4,4,4,58,5707797,58,7,215.464,4,1
4,18,5,1,3,5,5,5,58,5708630,43,1,218.385,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25135,942,8,6,18,\N,R,16,25,\N,24,14,186.905,6,0
25136,976,815,10,6,\N,R,15,39,\N,37,8,204.670,6,0
25137,1010,817,4,12,\N,R,19,28,\N,18,19,212.478,1,0
25138,1037,847,3,15,\N,R,19,9,\N,5,18,225.624,1,0


#### #3 and #4
adding circuitType and isHistoric from circuits.csv

In [30]:
#step 1, prep a trimmed version of races_df for left merging into merged2_df
races_df = races_df.drop(['round','name','date','time','url'],axis=1, inplace=False)

In [31]:
races_df

,raceId,year,circuitId
0,1,2009,1
1,2,2009,2
2,3,2009,17
3,4,2009,3
4,5,2009,4
...,...,...,...
1053,1069,2021,69
1054,1070,2021,32
1055,1071,2021,18
1056,1072,2021,77


In [32]:
#step 2, bring in circuitID from races.csv INTO merged2_df (refined results.csv) to prep the join
merged3_df = pd.merge(merged2_df, races_df, on='raceId', how='left')

In [96]:
merged3_df.head()

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,milliseconds,fastestLap,rank,fastestLapSpeed,familyStatus,Completion Status,year,circuitId
0,18,1,1,1,1,1,1,58,5690616,39,2,218.300,4,1,2008,1
1,18,2,2,5,2,2,2,58,5696094,41,3,217.586,4,1,2008,1
2,18,3,3,7,3,3,3,58,5698779,41,5,216.719,4,1,2008,1
3,18,4,4,11,4,4,4,58,5707797,58,7,215.464,4,1,2008,1
4,18,5,1,3,5,5,5,58,5708630,43,1,218.385,4,1,2008,1


In [ ]:
#step 3, left join the relevant columns into merged3_df from circuits.csv using circuitID as the joining column

In [34]:
#dropping the fluff out of circuits ahead of merge
circuits_df = circuits_df.drop(columns=['circuitRef','name','location','lat','lng','url'],axis=1,inplace=False)

In [35]:
circuits_df.head()

,circuitId,country,alt,isHistoric,trackType
0,23,Austria,678,0,0
1,29,Australia,58,0,2
2,64,Morocco,19,0,1
3,58,UK,20,0,1
4,1,Australia,10,0,2


In [36]:
#merging in circuits to merged3_df on circuitID
merge4_df = pd.merge(merged3_df,circuits_df,on='circuitId',how='left')

In [37]:
merge4_df

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,milliseconds,fastestLap,rank,fastestLapSpeed,familyStatus,Completion Status,year,circuitId,country,alt,isHistoric,trackType
0,18,1,1,1,1,1,1,58,5690616,39,2,218.300,4,1,2008,1,Australia,10,0,2
1,18,2,2,5,2,2,2,58,5696094,41,3,217.586,4,1,2008,1,Australia,10,0,2
2,18,3,3,7,3,3,3,58,5698779,41,5,216.719,4,1,2008,1,Australia,10,0,2
3,18,4,4,11,4,4,4,58,5707797,58,7,215.464,4,1,2008,1,Australia,10,0,2
4,18,5,1,3,5,5,5,58,5708630,43,1,218.385,4,1,2008,1,Australia,10,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25135,942,8,6,18,\N,R,16,25,\N,24,14,186.905,6,0,2015,69,USA,161,0,0
25136,976,815,10,6,\N,R,15,39,\N,37,8,204.670,6,0,2017,73,Azerbaijan,-7,0,2
25137,1010,817,4,12,\N,R,19,28,\N,18,19,212.478,1,0,2019,1,Australia,10,0,2
25138,1037,847,3,15,\N,R,19,9,\N,5,18,225.624,1,0,2020,13,Belgium,401,1,0


### #5, Adding in Driver Nationality

In [38]:
merge5_df = pd.merge(merge4_df,drivers_df[['driverId','nationality']],on='driverId',how='left')
#did it a different way here, merged just some select columns so I didn't need to do any drops

In [39]:
merge5_df

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,milliseconds,fastestLap,...,fastestLapSpeed,familyStatus,Completion Status,year,circuitId,country,alt,isHistoric,trackType,nationality
0,18,1,1,1,1,1,1,58,5690616,39,...,218.300,4,1,2008,1,Australia,10,0,2,British
1,18,2,2,5,2,2,2,58,5696094,41,...,217.586,4,1,2008,1,Australia,10,0,2,German
2,18,3,3,7,3,3,3,58,5698779,41,...,216.719,4,1,2008,1,Australia,10,0,2,German
3,18,4,4,11,4,4,4,58,5707797,58,...,215.464,4,1,2008,1,Australia,10,0,2,Spanish
4,18,5,1,3,5,5,5,58,5708630,43,...,218.385,4,1,2008,1,Australia,10,0,2,Finnish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25135,942,8,6,18,\N,R,16,25,\N,24,...,186.905,6,0,2015,69,USA,161,0,0,Finnish
25136,976,815,10,6,\N,R,15,39,\N,37,...,204.670,6,0,2017,73,Azerbaijan,-7,0,2,Mexican
25137,1010,817,4,12,\N,R,19,28,\N,18,...,212.478,1,0,2019,1,Australia,10,0,2,Australian
25138,1037,847,3,15,\N,R,19,9,\N,5,...,225.624,1,0,2020,13,Belgium,401,1,0,British


# Data Wrangling / Fixing Nulls Work
## 1 - fixing milliseconds (there are a lot of nulls)

### Plan -

#### Step 1 - Drop the old Milliseconds from merged5_df and add the new milliseconds (from our new df called summed_df)

#### Step 2 - Take all Laptimes (in milliseconds) from LapTimes_df and sum them for each driver in each race, thus replicating the total time column (this does NOT factor in laps completed, so even if a driver completes just one lap, they will still have a time)


##### Key Note - Drivers who don't complete a lap at all (vehicle failure at start-line) will still have a Null, there are approx. 217 of these)

In [40]:
merge5_df = merge5_df.drop(columns=['milliseconds'],axis=1,inplace=False)

In [42]:
merge5_df.describe()

,raceId,driverId,constructorId,grid,positionOrder,laps,familyStatus,Completion Status,year,circuitId,alt,isHistoric,trackType
count,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000,25140.000000
mean,516.215712,249.500438,47.323230,11.211734,12.942164,45.804455,4.107399,0.547971,1989.378123,22.074105,277.722753,0.312729,0.429395
std,289.167348,256.875639,58.208313,7.275871,7.749181,30.036217,1.377162,0.497703,18.880974,16.616505,402.379398,0.463614,0.802649
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1950.000000,1.000000,-7.000000,0.000000,0.000000
25%,286.000000,56.000000,6.000000,5.000000,6.000000,21.000000,4.000000,0.000000,1976.000000,9.000000,18.000000,0.000000,0.000000
50%,501.000000,158.000000,25.000000,11.000000,12.000000,52.000000,4.000000,1.000000,1990.000000,18.000000,153.000000,0.000000,0.000000
75%,759.000000,347.000000,57.000000,17.000000,19.000000,66.000000,5.000000,1.000000,2006.000000,32.000000,401.000000,1.000000,0.000000
max,1060.000000,854.000000,214.000000,34.000000,39.000000,200.000000,6.000000,1.000000,2021.000000,76.000000,2227.000000,1.000000,2.000000


In [19]:
summed_lapTimes = lap_times_df.groupby(['raceId','driverId']).agg({'milliseconds':'sum'})

In [43]:
merge6_df = pd.merge(merge5_df,summed_lapTimes, on=['raceId','driverId'], how='right')
#Key Note - I did an inner merge here because again, we are confident that there are no nulls in raceID or driverID
#When I attempted this using a left join on accident, we got numerous nulls in a variety of columns.

In [76]:
merge6_df.rename(columns={'milliseconds':'total_lap_time'}, inplace= True)
merge6_df

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,fastestLap,rank,...,familyStatus,Completion Status,year,circuitId,country,alt,isHistoric,trackType,nationality,total_lap_time
0,1,1,1,18,\N,D,20,58,39,13,...,3,0,2009,1,Australia,10,0,2,British,5658698
1,1,2,2,9,10,10,10,58,48,5,...,4,1,2009,1,Australia,10,0,2,German,5662869
2,1,3,3,5,6,6,6,58,48,1,...,4,1,2009,1,Australia,10,0,2,German,5661506
3,1,4,4,10,5,5,5,58,53,9,...,4,1,2009,1,Australia,10,0,2,Spanish,5660663
4,1,6,3,11,\N,R,18,17,6,18,...,1,0,2009,1,Australia,10,0,2,Japanese,1560978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9461,1060,847,3,8,11,11,11,70,56,16,...,4,1,2021,70,Austria,678,0,0,British,5051219
9462,1060,849,3,18,16,16,16,70,62,11,...,4,1,2021,70,Austria,678,0,0,Canadian,5077844
9463,1060,852,213,7,12,12,12,70,54,9,...,4,1,2021,70,Austria,678,0,0,Japanese,5056962
9464,1060,853,210,20,19,19,19,69,49,19,...,4,1,2021,70,Austria,678,0,0,Russian,5091577


## Creating Tables of Average Race times (milliseconds) and minimum laptimes (milliseconds) for each driver in each race

#### Average Laptime

In [78]:
average_lapTime = lap_times_df.groupby(['raceId','driverId'], as_index = False).agg({'milliseconds':'mean'})
average_lapTime.rename(columns={'milliseconds':'average_lap_time'}, inplace= True)

#### Minimum Lap Time

In [79]:
min_lapTime = lap_times_df.groupby(['raceId','driverId'], as_index = False).agg({'milliseconds':'min'})
min_lapTime.rename(columns={'milliseconds':'minimum_lap_time'}, inplace= True)

In [64]:
min_lapTime.head()

milliseconds
raceId driverId              
1      1                89020
       2                88283
       3                87706
       4                88712
       6                89923

In [138]:
average_race_time.isna().sum()

milliseconds    0
dtype: int64

#### Bringing it all together now

In [80]:
merge7_df = pd.merge(merge6_df,average_lapTime, on=['raceId','driverId'])

In [72]:
merge7_df

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,fastestLap,rank,...,Completion Status,year,circuitId,country,alt,isHistoric,trackType,nationality,milliseconds_x,milliseconds_y
0,1,1,1,18,\N,D,20,58,39,13,...,0,2009,1,Australia,10,0,2,British,5658698,97563.758621
1,1,2,2,9,10,10,10,58,48,5,...,1,2009,1,Australia,10,0,2,German,5662869,97635.672414
2,1,3,3,5,6,6,6,58,48,1,...,1,2009,1,Australia,10,0,2,German,5661506,97612.172414
3,1,4,4,10,5,5,5,58,53,9,...,1,2009,1,Australia,10,0,2,Spanish,5660663,97597.637931
4,1,6,3,11,\N,R,18,17,6,18,...,0,2009,1,Australia,10,0,2,Japanese,1560978,91822.235294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9461,1060,847,3,8,11,11,11,70,56,16,...,1,2021,70,Austria,678,0,0,British,5051219,72160.271429
9462,1060,849,3,18,16,16,16,70,62,11,...,1,2021,70,Austria,678,0,0,Canadian,5077844,72540.628571
9463,1060,852,213,7,12,12,12,70,54,9,...,1,2021,70,Austria,678,0,0,Japanese,5056962,72242.314286
9464,1060,853,210,20,19,19,19,69,49,19,...,1,2021,70,Austria,678,0,0,Russian,5091577,73790.971014


In [81]:
merge8_df = pd.merge(merge7_df,min_lapTime, on=['raceId','driverId'])

In [84]:
merge8_df = merge8_df.replace(r'\N', np.NaN)

In [85]:
merge8_df.describe()

,raceId,driverId,constructorId,grid,positionOrder,laps,familyStatus,Completion Status,year,circuitId,alt,isHistoric,trackType,total_lap_time,average_lap_time,minimum_lap_time
count,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9466.000000,9.466000e+03,9466.000000,9466.000000
mean,500.169977,249.438411,36.706634,11.070357,10.817135,52.982252,4.071097,0.750898,2008.727129,16.667441,199.641242,0.297380,0.476231,5.060097e+06,98030.991468,90887.825903
std,408.988287,355.593273,63.937258,6.240870,6.043638,17.737604,1.069539,0.432516,7.241252,17.218570,307.978602,0.457129,0.851905,1.689662e+06,21398.052922,13743.675973
min,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1996.000000,1.000000,-7.000000,0.000000,0.000000,8.657900e+04,62932.344828,55404.000000
25%,121.000000,15.000000,4.000000,6.000000,6.000000,49.000000,4.000000,1.000000,2002.000000,6.000000,10.000000,0.000000,0.000000,4.846332e+06,85102.377976,80849.000000
50%,236.000000,35.000000,9.000000,11.000000,11.000000,56.000000,4.000000,1.000000,2009.000000,13.000000,109.000000,0.000000,0.000000,5.483886e+06,94761.084615,89008.500000
75%,934.000000,810.000000,20.000000,16.000000,16.000000,66.000000,4.000000,1.000000,2015.000000,20.000000,228.000000,1.000000,0.000000,5.858067e+06,106240.737981,99527.750000
max,1060.000000,854.000000,214.000000,24.000000,24.000000,87.000000,6.000000,1.000000,2021.000000,76.000000,2227.000000,1.000000,2.000000,1.476231e+07,753493.000000,446036.000000


In [86]:
merge8_df.isna().sum()

raceId                  0
driverId                0
constructorId           0
grid                    0
position             2164
positionText            0
positionOrder           0
laps                    0
fastestLap           2745
rank                 2704
fastestLapSpeed      2745
familyStatus            0
Completion Status       0
year                    0
circuitId               0
country                 0
alt                     0
isHistoric              0
trackType               0
nationality             0
total_lap_time          0
average_lap_time        0
minimum_lap_time        0
dtype: int64

In [90]:
merge8_df.to_csv("./data/processed/MasterData2.csv", index = False)

## Normalizing Milliseconds

#### Step 1 - Drop all rows where Laps = 0 (Car failure at race-start) 

#### Step 2 - Divide Each Driver/Race Milliseconds by the Laps completed column to produce an average lap time

In [141]:
zero_lap_cut_merge6 = merge6_df[merge6_df.laps !=0]

In [143]:
zero_lap_cut_merge6

,raceId,driverId,constructorId,grid,position,positionText,positionOrder,laps,fastestLap,rank,...,familyStatus,Completion Status,year,circuitId,country,alt,isHistoric,trackType,nationality,milliseconds
0,18,1,1,1,1,1,1,58,39,2,...,4,1,2008,1,Australia,10,0,2,British,5690616
1,18,2,2,5,2,2,2,58,41,3,...,4,1,2008,1,Australia,10,0,2,German,5696094
2,18,3,3,7,3,3,3,58,41,5,...,4,1,2008,1,Australia,10,0,2,German,5698779
3,18,4,4,11,4,4,4,58,58,7,...,4,1,2008,1,Australia,10,0,2,Spanish,5707797
4,18,5,1,3,5,5,5,58,43,1,...,4,1,2008,1,Australia,10,0,2,Finnish,5708630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9461,942,8,6,18,\N,R,16,25,24,14,...,6,0,2015,69,USA,161,0,0,Finnish,3068772
9462,976,815,10,6,\N,R,15,39,37,8,...,6,0,2017,73,Azerbaijan,-7,0,2,Mexican,6326176
9463,1010,817,4,12,\N,R,19,28,18,19,...,1,0,2019,1,Australia,10,0,2,Australian,2599322
9464,1037,847,3,15,\N,R,19,9,5,18,...,1,0,2020,13,Belgium,401,1,0,British,1025778
